###  Importing Library

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR


#### Hyperparameters

In [2]:
batch_size = 64
image_size = 64
nz = 100
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
num_epochs = 10
r1_gamma = 10.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Memeriksa apakah CUDA tersedia
print(torch.cuda.is_available())

# Jika True, berarti GPU Anda bisa digunakan oleh PyTorch
# Nama GPU (untuk device index 0)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.version.cuda)


True
NVIDIA GeForce GTX 1060 6GB
11.8


# 2. Dataset & Augmentations

In [3]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Ganti path di bawah sesuai lokasi dataset Anda
data_dir = r"C:\Users\dawwi\Downloads\Dataset 4"
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# 3. Definisi Model DCGAN (Generator & Discriminator)

In [4]:
class Generator(nn.Module):
    def __init__(self, nz, ngf=64, nc=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, x):
        return self.main(x)

In [5]:
class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  # DCGAN standard
        )
    def forward(self, x):
        return self.main(x)

netG = Generator(nz).to(device)
netD = Discriminator().to(device)

# 4. Loss, Optimizer, Scheduler

In [6]:
criterion = nn.BCELoss()

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))

# Scheduler: menurunkan LR setengah setiap 10 epoch
schedulerD = StepLR(optimizerD, step_size=10, gamma=0.5)
schedulerG = StepLR(optimizerG, step_size=10, gamma=0.5)

# Logging
lossD_history = []
lossG_history = []

fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)

# 5. Training Loop (with R1 Regularization)

In [7]:
#Logging Loss *per-epoch*
lossD_epoch = []
lossG_epoch = []

In [ ]:
for epoch in range(num_epochs):
    epoch_lossD = 0.0
    epoch_lossG = 0.0
    total_batches = 0

    for i, (imgs, _) in enumerate(dataloader):
        b_size = imgs.size(0)
        total_batches += 1

        real_imgs = imgs.to(device)

        # --------------------------------------
        # (A) Train Discriminator
        # --------------------------------------
        netD.zero_grad()

        # Real path
        real_imgs.requires_grad_(True)
        label_real = torch.full((b_size,), 1.0, dtype=torch.float, device=device)
        output_real = netD(real_imgs).view(-1)
        lossD_real = criterion(output_real, label_real)
        lossD_real.backward(retain_graph=True)

        # R1 penalty
        grad_real = torch.autograd.grad(
            outputs=output_real.sum(),
            inputs=real_imgs,
            create_graph=True
        )[0]  # shape: (b_size, 3, 64, 64)
        r1_penalty = r1_gamma * grad_real.pow(2).view(b_size, -1).sum(1).mean()
        r1_penalty.backward()

        real_imgs.requires_grad_(False)

        # Fake path
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_imgs = netG(noise)
        label_fake = torch.full((b_size,), 0.0, dtype=torch.float, device=device)
        output_fake = netD(fake_imgs.detach()).view(-1)
        lossD_fake = criterion(output_fake, label_fake)
        lossD_fake.backward()

        lossD_total = lossD_real + lossD_fake + r1_penalty
        optimizerD.step()

        # --------------------------------------
        # (B) Train Generator
        # --------------------------------------
        netG.zero_grad()
        label_fake_forG = torch.full((b_size,), 1.0, dtype=torch.float, device=device)
        output_fake_forG = netD(fake_imgs).view(-1)
        lossG = criterion(output_fake_forG, label_fake_forG)
        lossG.backward()
        optimizerG.step()

        epoch_lossD += lossD_total.item()
        epoch_lossG += lossG.item()

    # Rata-rata Loss per epoch
    avg_lossD = epoch_lossD / total_batches
    avg_lossG = epoch_lossG / total_batches
    lossD_epoch.append(avg_lossD)
    lossG_epoch.append(avg_lossG)

    # Step the scheduler each epoch
    schedulerD.step()
    schedulerG.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] | LossD: {avg_lossD:.4f} | LossG: {avg_lossG:.4f} | "
          f"LR_D: {schedulerD.get_last_lr()[0]:.6f} | LR_G: {schedulerG.get_last_lr()[0]:.6f}")

    # Save sample image di setiap epoch
    with torch.no_grad():
        fake_sample = netG(fixed_noise).detach().cpu()
    grid = make_grid(fake_sample, nrow=8, normalize=True)
    os.makedirs("samples", exist_ok=True)
    save_image(grid, f"samples/epoch_{epoch+1}.png")
    print(f"Sample image saved: samples/epoch_{epoch+1}.png")

print("Training Finished.")

# 6. Plot Loss vs. Iteration

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training Loss (D & G) over Iterations")
plt.plot(lossD_history, label="LossD")
plt.plot(lossG_history, label="LossG")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.savefig("training_loss_plot.png")
plt.show()
print("Plot saved: training_loss_plot.png")

# 7. Plot Loss vs. Epoch

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training Loss (D & G) per Epoch")
plt.plot(lossD_epoch, label="LossD (Epoch)")
plt.plot(lossG_epoch, label="LossG (Epoch)")
plt.xlabel("Epoch")
plt.ylabel("Average Loss")
plt.legend()
plt.savefig("training_loss_epoch_plot.png")
plt.show()

print("Plot saved: training_loss_epoch_plot.png")